# Transfer data between instances

Sometimes, we want to load one instance just for reading data and use another to write data.

This is for instance, relevant if we only have read access to an instance.

You can do this with LaminDB via the `using` parameter.

## Setup

In [ ]:
!lamin init --storage ./test-transfer --schema bionty

In [ ]:
import lamindb as ln

ln.settings.verbosity = "info"

In [ ]:
ln.track()

## Query cellxgene-census

Query a dataset from the public `cellxgene-census` instance:

In [ ]:
dataset = ln.Dataset.filter(
    using="laminlabs/cellxgene-census",
    reference="c888b684-6c51-431f-972a-6c963044cef0",
).one()
dataset

Save the dataset to our default instance:

In [ ]:
dataset.save()

Let's look at the database content:

In [ ]:
ln.view()

In [ ]:
dataset.ulabels.df()

In [ ]:
!lamin delete --force test-transfer